In [1]:
! pip3 install transformers
! pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TrainerCallback, TextDataset, DataCollatorForLanguageModeling, TrainingArguments, Trainer
import torch
from os.path import join


DEVICE='cuda'


def get_ru_model(checkpoint = "sberbank-ai/rugpt3small_based_on_gpt2"):
    model = AutoModelForCausalLM.from_pretrained(checkpoint).to(DEVICE)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    return model, tokenizer


def print_generations(prefix = ". ", n_samples=4, max_len=70, one_line=True):
    global model, tokenizer
    input_ids = tokenizer.encode(prefix, return_tensors='pt').to(DEVICE)
    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=max_len, 
        top_k=50,
        top_p=0.90, 
        num_return_sequences=n_samples,
        pad_token_id=50256
    )
    for row in sample_outputs:
        text = tokenizer.decode(row)
        if one_line:
            text = text.split("\n")[0]
        print(text)


class PringSamplingCallback(TrainerCallback):
    def on_log(self, args, state, control, **kwargs):
        print("=====", "progress:", state.epoch, "=========")
        print_generations()
        print("==========================")


def freeze_layers(model, n_layers=8):
    assert (n_layers >=0) and (n_layers <= 12)
    to_freeze = (
        model.transformer.wte,
        model.transformer.wpe,
        model.transformer.drop,
        model.transformer.h[:n_layers]
    )
    for paramgroup in to_freeze:
        for param in paramgroup.parameters():
            param.requires_grad = False


def finetune(model, tokenizer, path, n_epochs="auto", adjust_to_size=5e6):
    if n_epochs == "auto":
        filesize = len(open(path).read())
        num_train_epochs = adjust_to_size / filesize
        print("SETTING N_EPOCHS TO ", num_train_epochs)
    else:
        num_train_epochs = n_epochs
    
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = path,
        block_size = 256,
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm = False,
    )
    training_args = TrainingArguments(
        output_dir=".",
        overwrite_output_dir=False,
        per_device_train_batch_size=8,
        num_train_epochs=num_train_epochs,
        logging_steps=50,
        save_steps=500,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
        callbacks=[PringSamplingCallback],
    )
    trainer.train()
    trainer.save_model()


COLAB_DATA_PATH = "/content/gdrive/MyDrive/datasets/"
OLDRU_PATH = join(COLAB_DATA_PATH, "oldrussian.txt")
DOST_PATH = join(COLAB_DATA_PATH, "dost.txt")
TOLST_PATH = join(COLAB_DATA_PATH, "tolst.txt")
LENIN_PATH = join(COLAB_DATA_PATH, "lenin.txt")

# Тексты на древнерусском языке

In [4]:
model, tokenizer = get_ru_model()
freeze_layers(model, 4)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
finetune(model, tokenizer, OLDRU_PATH)

SETTING N_EPOCHS TO  0.587214714943076


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11776
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 865
  Number of trainable parameters = 56704512


Step,Training Loss
50,4.419300
100,4.291400
150,4.219200
200,4.155000
250,4.096700
300,4.071500
350,4.076700
400,4.057100
450,4.059800
500,4.016800


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.033967391304347824 =========
.  Он же, видя что питием бо и от чадъя и от духовъ приимъ. о семъже поели со мною, и крѣпие дѣти въ церковь, а и прииде, и бысть съ пнемъ сь в церковь, и въ ней
.  Вѣрное же преставление, яко всѣм бо, якоже вѣдущие., възможенъ житию, тѣла и мысли, бысть, бѣяшшу, вѣтмо мѣсяца, и паки въздержанишися.
.  Когда же досточны и отчисто их сѣдили, и оных же съсоръзницъ наѣх и снѣкъ при нихъ, и отъ нихъ же съградоша съ нимиъ, и от нихъ же въ тъмъ же тъмъ же в
.  Он же от лица свѣта сьде на лѣта, и мѣськи постави, и от лѣта, и от тѣхъ, но и от нихъ. и яко въ ту ночь от нихъ къ царю и от царя к нему, и на то время къ его. 


...

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.5774456521739131 =========
.  А также до вечера, понеже възвещаю, и в немь будуть, аще ли въмьчь, тогда имѣемъ внидоша въ кыевъ. от нихъ же и доиде в цесарьскиа, на то же время убо не преже вси
.  аще есть в чем не тѣмо, аще же есть, но есть в чем, и се сътворити възбратиша, якоже и до сихъ. точию и въ всѣхъ еси. аще же есть, или не въста еси; вѣдаеши ли
.  Яже есть по земли, и в ней есть мѣсто, и мѣсто, иже елли его отьця, и съмь, якоже елли, в нейже, бѣса, и даждь и до гроба вниди. и се есть мѣсто, и в
.  И на свѣтѣ вѣмъ вѣрѣхъ имѣя, тако иже, иже въ мѣсту христосъ, яко не вървутъ тѣло, не възвысятъ от земли, на землю, не възъгохомъ, не




Training completed. Do not forget to share your model on huggingface.co/models =)


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.5876358695652174 =========
.  Да, рече о них, яко многа к нимъ приити. се бо имѣния.  и приидоша ко граду. и пришедша ко граду крѣпцы, и взиде на мѣсто имаша, и приидоша к граду; и поставиша городъ и
.  И в томъ часѣ не быша от мала, ни великаго чюдеса. и на томъ часѣ не бысть. и на томъ часѣ не бысть. 
. 
.  и тако възрасту въ блаженной,


Saving model checkpoint to .
Configuration saved in ./config.json
Configuration saved in ./generation_config.json
Model weights saved in ./pytorch_model.bin


In [6]:
print_generations(prefix="самое главное это", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



самое главное это вѣрие,
самое главное это воя, а иежеи внидуще в немъ.
самое главное это слово». иже от того времени ина единаго, якоже от бога о сем есть, иже и сам бог да не возненавидят, да и не возненавидят.
самое главное это


In [7]:
print_generations(prefix="зачем", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



зачем и кто и как? не измѣютъ, да и какъ будетъ, не знают, а кто и какъ будетъ, а кто и какъ будетъ, а кто и какъ будутъ, то не ведаютъ.
зачемъ сѣче его на пути, сыйже въ агианѣ, и иже поставиши на пути его». и, не въздержавши, паки внидоша и сѣче сѣче на пути его, и иже постави его на пути его, и паки положити его на пути его и спяти ему от него, аки въсконь его питися. и тако прииде вѣтренъ, и дошед до земли, овии же, паче всехъ своих, и рече: «время, отьцю. тако и вы, отче, на земле живя, каяйтесь и молим тя. да будеши ны». и бывъши сѣчу, приидох до земли, и рече, — глаголаше господь: «видѣх васъ, въздаша ми тя». и пре
зачемъ, но, аще убоится имѣния, имѣющу бога вѣра его. вѣрь ми велика, в нейже азъ от него, яко ничию же и мняше, ни сѣдою, ни сѣднаго, яко богъ, сътворять. яко же не от бога, а от бога от бога и отъ
зачем ми, яко же быти, или, яко не хотѣл быти, тако и быти ми не вѣмь, и тѣмь жити на мѣстехъ.


In [8]:
print_generations(prefix="когда", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



когда от нее, оже глаголаше, оже глаголѣти: «съдѣте на небо и повѣдайте, яко дѣло и воля ваша суть, егоже бо есть богъ. и възбранно и христолюбезное его есть иже есть есмы, оже не вѣру от насъ творяху, ни единому другому же с вами не обрѣтуя. да аще бо вамъ чего имать на небо и повѣдати, не хто ли ми вѣдаю вамь и есмы, аще бо вамъ о томъ есмы, и не хто ли ми вѣжду?».
когда онъ же сядя и рече к ней: «видѣ, брате, что нѣсть поѣда в той градѣ и нѣ что есть въ немъ? или нѣкогда? и есть ли у тебя вѣсты, а у насъ съ тобою вѣстники и дѣла дѣяти и дѣлати, и дѣлати есть приати въ рускыихъ городѣхъ? и не смѣемъ ити. и аще и нѣсмь кто достоитъ на немъ, то, брате, от насъ мняшть с собою крѣпкие дѣли». она же оттудяше и повѣда ему, како есть и каковы дѣти, а самъ приати в русь, а самъ въ русь. и оттуду же бѣше у нее в русь. она же убо,
когда у нас много народу, от вас много и ты, не жалей о нем, а ты молиши бога и нас, чтобы тебя и нас на мя не обижали! а ты, брат, ни у кого не бѣдай, а у нас мало,

# Собрание сочинений Достоевского

In [9]:
model, tokenizer = get_ru_model()
freeze_layers(model, 8)
finetune(model, tokenizer, DOST_PATH)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/f2f7c585b05a16726efe8974586e10b4d5939082/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.2

SETTING N_EPOCHS TO  0.34442475176791504


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file /content/gdrive/MyDrive/datasets/cached_lm_GPT2TokenizerFast_256_dost.txt [took 0.940 s]
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch 

Step,Training Loss
50,3.661700
100,3.637900
150,3.634500
200,3.578700
250,3.589500
300,3.579500
350,3.562600
400,3.549300
450,3.542200
500,3.570900


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



...

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.32894736842105265 =========
.  В начале тридцатых годов в это же самое время, в сороковых, к великому удовольствию немцев, в Петербурге было уже много и в то же самое время много всего.  К этой теме, в том же году, при жизни моего друга, который был в ссылке, о которых сейчас уже очень и очень хорошо известно, принадлежал он,
.  Все-таки надо дать ему время и пройти через это.  А если и пропустим, то зачем нам в это дело вся эта грязь, чтобы потом не идти к нему, чтобы не быть ему не на своих же ногах, а к другому человеку, к чему, к чему идти с такой грязью?  Вот и получается что-то
.  В этих словах, с виду как будто наигранная улыбка и все-таки в лице была улыбка, была какая-то странная, удивительная, почти до отвращения простая, в общем-то, и в то же время была какая-то необыкновенная, странная, словно бы какая-то волшебная. 
.  Да и вообще, я в таком смысле не вожусь, – сказал он. – Вот мне кажется, у вас есть один очень серьезный и очень важный вопрос.




Training completed. Do not forget to share your model on huggingface.co/models =)


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.34484649122807015 =========
.  Мне показалось, что тут какое-то что-то было другое.  И в особенности, что-то не похожее на правду. 
.  И хотя, по сути своей, она не была похожа ни на какую другую женщину; но что-то во взгляде, взгляде, в выражении глаз, в каждом движении этого лица, в каждом движении его головы и в каждом движении этих движений было что-то странное.  Оно как будто не отличалось от обыкновенных людей ни в чем.
.  Огородная часть, как всегда, довольно своеобразна.  В основном это дома крестьян.  Захоронения же можно наблюдать за частными, на случай, если вы проживаете в другой деревне.  Сходные деревянные строения тоже вполне могут быть похожи на дворы крестьян.  В этих местах есть еще и сельские избы,
.  По окончании работы необходимо сообщить в комиссию по делам несовершеннолетних, и тогда уже и мы сами будем в полном порядке и совершенно по закону. 


Saving model checkpoint to .
Configuration saved in ./config.json
Configuration saved in ./generation_config.json
Model weights saved in ./pytorch_model.bin


In [10]:
print_generations(prefix=". Самое главное это", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



. Самое главное это – чтоб все-таки и в ту же минуту не случилось. «Нет, – подумал Павел Павлович, – он не может быть в эту минуту так бессилен, что не смеет переступить через себя». Он подошел к кровати и стал в изголовье, как бы обдумывая какое-то решение. Он не знал, в какую сторону броситься, чтоб не только убежать, но и спастись. Он уже приготовился бежать, но как будто бы вдруг как бы что-то изменилось, что-то исчезло в нем. Что это было? Почему он вдруг опять ничего не видит? Что он видел?
. Самое главное это не в том, что это будет, это не то, что это будет; а именно в том, что я уже говорил, то, чего я не могу сказать. В том, что я еще не знаю, что это будет. Но, может быть, и в том, что я это все понимаю, и если я не говорю этого, это, вероятно, не есть, но то, что я говорю, если я не говорю этого, я все-таки говорю это. Это было бы слишком неубедительно, потому что я и так знаю все, но ведь и так было бы довольно, даже слишком легко и просто; ибо как человек, который много з

In [11]:
print_generations(prefix="В 2024 году", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



В 2024 году (в связи с «Сонмами») был произведен ремонт дома на Рублёвке, но не в здании, а в частном доме на Старом Арбате. Не знаю почему, да и зачем так делать, у меня в том же доме ремонт был, а я и не помню его, так что это скорее к слову… А до тех пор было дело, и я не знаю как в этот дом попасть.
В 2024 году в России по предложению В. И. Ленина в письме к одному из знакомых в Москве «о новых вещах», он был поставлен в один ряд с представителями духовенства и с некоторыми из приближенных, бывших еще в то время членами правительства, с некоторыми из которых он и сам встречался иногда, например, еще в Петербурге. В письмах к В. И.
В 2024 году, после смерти графа Разумовского, этот молодой человек в своем кабинете, над письменным столом, продолжал писать. Так как же мог он так много писать, то, конечно, не мог не подумать, что его непременно убьют. Он так и сказал вслух, что он и есть убийца и убийца.
В 2024 году этот период становится для крестьянским юношей самым трудным. Он еще н

In [12]:
print_generations(prefix="Больше всего на свете я", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



Больше всего на свете я ее боюсь, – отвечала она тихо и нежно. – Я в нее хочу превратиться и стать ее матерью. И я в это верю, потому что… я… Я, наконец, она, я… я… Я…
Больше всего на свете я хотела быть его… как будто это так и было. Убить…
Больше всего на свете я в восторге! Это было одно из моих самых счастливых воспоминаний! Я была в восторге. Она была счастлива, в восторге и даже, кажется, чуть в слезы не билась, но я чувствовала, что она плачет. Когда она выходила, она уже была в волнении. У меня были дрожки в коленях. Как мне хотелось ее поцеловать, но она сама мне все целовала, а я только в недоумении глядела на нее… Она целовала меня во весь рот, потом вдруг, в порыве благодарности, стала ее целовать, целовать на глазах ее блестели слезы, и так она в упоении целовала меня всю ночь! – И она покраснела.
Больше всего на свете я ненавидел эти деньги. В тот же вечер я принес эту вещь. Я положил ее на стол, положил, открыл… я взял ее… и, клянусь, мне показалось… показалось, что я… в

Собрание сочинений Толстого

In [13]:
model, tokenizer = get_ru_model()
freeze_layers(model, 8)
finetune(model, tokenizer, TOLST_PATH)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/f2f7c585b05a16726efe8974586e10b4d5939082/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.2

SETTING N_EPOCHS TO  0.8218274747444199


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file /content/gdrive/MyDrive/datasets/cached_lm_GPT2TokenizerFast_256_tolst.txt [took 0.704 s]
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch

Step,Training Loss
50,3.506400
100,3.446400
150,3.433100
200,3.426200
250,3.390500
300,3.433600
350,3.358800
400,3.383300
450,3.374000
500,3.364800


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



...

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.796812749003984 =========
.  В начале лета он вернулся в свои родные места, оставив мать и сестру. 
.  И то, что он хотел сказать, было правдой, но для меня так и осталось тайной, я не знал тогда, что было бы лучше, если бы он сказал, что это хорошо, что он не любит меня, но, видимо, думал, что не понимает, потому что я люблю его.
.  В этих словах, с которым и в первый раз в первый раз в его жизни, он был не только поражен словами, но и совершенно серьезно удивлен тому, как точно так же он понимал их и знал, в особенности, если он верил, что им удастся все это сказать.  Он только теперь понял всю глубину значения того, что он
.  Да и не только, в таком смысле, в нем много общего, как и в моем прошлом.  Я много думала о том, как, и что нужно делать, и что я для этого сделаю, что бы сделать для того, чтобы быть, как человек, я не только люблю, я люблю, люблю то, что есть и чем




Training completed. Do not forget to share your model on huggingface.co/models =)


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.8220451527224436 =========
.  При этом необходимо учитывать, что у каждого человека бывают свои индивидуальные причины (высокая или маленькая температура воздуха, головная боль, плохое самочувствие). 
.  С помощью этих советов вы не только избавитесь от боли в желудке и поднимите его аппетит, но и будете знать, как в домашних условиях устранить эту неприятную проблему. 
. 
.  Она сидела в глубоком кресле, лицом к лицу с Петром.  По тому, как она старалась подавить страх, он заметил, что она пристально, внимательно следит за каждым его словом.  Она молчала.  Он, в том же положении, в которое его она поставила, знал, что она не захочет говорить ни о чем и потому смотрел на


Saving model checkpoint to .
Configuration saved in ./config.json
Configuration saved in ./generation_config.json
Model weights saved in ./pytorch_model.bin


In [14]:
print_generations(prefix="В 2024 году", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



В 2024 году в Петербурге в Петербурге был построен новый типовой дом для петербургского купечества. Князь Дмитрий, не дождавшись разрешения войти в него, вошел с женой в свою спальню и при участии супруги. В ней в комнате в эту минуту был и сын князя Дмитрия, который был в это время один и с матерью и с дядей и, вместе с своею дочерьми, в спальне в комнате княжны. Князь Дмитрий, оставшись в одиночестве в комнате и не смея войти в нее, с своей женой, в это время находился в комнате княжны.
В 2024 году Россия пережила первую Великую войну, когда русский флот в виде эскадр дошел до Парижа в 20-х. Русская дипломатия была занята во Франции.
В 2024 году, когда она была сделана, ее и причислили к лику святых. После этого в 1812 году она была перевезена в Париж, где была встречена и положена в алтарь, названный в ее честь церковью святого Доминика».
В 2024 году, между 2026 и 2026 годами, в составе царского войска был объявлен план об уничтожении русского царя, его семьи и всего русского войска

# Собрание сочинений Ленина

In [15]:
model, tokenizer = get_ru_model()
freeze_layers(model, 6)
finetune(model, tokenizer, LENIN_PATH)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/f2f7c585b05a16726efe8974586e10b4d5939082/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.2

SETTING N_EPOCHS TO  0.14183550978504744


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file /content/gdrive/MyDrive/datasets/cached_lm_GPT2TokenizerFast_256_lenin.txt [took 1.256 s]
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch

Step,Training Loss
50,3.389900
100,3.275100
150,3.276700
200,3.275400
250,3.212600
300,3.197000
350,3.145900
400,3.146000
450,3.209800
500,3.172900


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



...

Model weights saved in ./checkpoint-500/pytorch_model.bin
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.13496932515337423 =========
.  И мы с женой тоже на них ссылаемся, а потому и сами на них ссылаемся.  А теперь мы с женой на нее ссылаемся, а мы – на нее.  И, если у нас нет никакого намерения, то пусть все, кто на нас ссылается, не обижаются.  Так почему же они должны обижаться
.  Но не все они могут быть правы; как не всегда можно и не всегда нужно иметь дело с правыми, правыми или правыми, – вот в чем, кажется, суть их взаимоотношений. И вот как обстоит дело с правомерностью того или другого? Что является правомерным в правом поведении этих политических партий? Это вопрос в
.  Это доказывает тот факт, что не только в России и во Франции, и в Германии, и в Италии, и во Франции, что нет ни одного государства, которое не было бы уничтожено и уничтожено по частям или по частям.  Не может быть ни одного государства, где нет таких крупных капиталистических предприятий, где капиталистическая революция не
.  В начале 1906 года Ленин писал письмо, в котором писал: "Поражает



Training completed. Do not forget to share your model on huggingface.co/models =)


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



===== progress: 0.14184049079754601 =========
.  Однако для нас, всех этих групп, мы должны стремиться к тому, чтобы не только в виде исключения, но и в виде исключения из нашей политической партии, в партии, которая бы представляла собой более сплоченную и сплоченную группу.Наоборот, это может идти вразрез с политической необходимостью, может быть, с экономической потребностью, или же
.  В сущности, это даже и в голову не приходило, что на деле, хотя бы и теоретически, большевики являются демократами, которые ведут борьбу с революцией.  К. Маркс, например, доказывал, что только так можно провести социалистическую революцию, а затем и победу революции. 
.  За свою свободу он и в революцию не стал, но дело его, как говорят, в том, что он не стал говорить об этом, а говорил о политике. 
.  А в самом деле, что такого в нашей статье в пользу революционного правительства?  На что я не должен и никогда не должен смотреть, чтобы мы, большевики, думали, что дело партии пойдет на пользу социали

Saving model checkpoint to .
Configuration saved in ./config.json
Configuration saved in ./generation_config.json
Model weights saved in ./pytorch_model.bin


In [16]:
print_generations(prefix=". самое важное", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



. самое важное, что мы можем делать, это создавать что бы то ни было более или менее в форме, когда-либо, в форме движения, или иначе, в форме организации.
. самое важное, чего хочет достигнуть буржуазный народ в этом столетии, – это перехода к демократии. И именно поэтому – борьба с буржуазной демократией, как бы она ни была вредна для буржуазии в России, как бы она ни была вредна для всех угнетенных наций в мире. Но именно благодаря этому переходу в России может теперь быть и буржуазная демократия. Но в этой демократии не должны быть неучтены и те условия, о которых говорит буржуазия. – В России нет ни одного сколько-нибудь известного демократического принципа, по которому нельзя было бы строить демократию. И здесь – только одна гарантия, что буржуазия в России не должна будет превращаться в какую-нибудь одну политическую партию. Но при всем разграничении с буржуазией в России можно добиться и этого. Борьба с буржуазной демократией – это – только первая ступень. Вторая ступень – это 

In [17]:
print_generations(prefix=". я верю, что", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



. я верю, что мы, – сказал Фокс, не отрываясь от окна. – Мы должны быть готовы защищать себя сами. Мы не хотим ни о какой защите ни просить, ни просить, а должны бороться со всеми этими противниками. Но только мы не хотим, чтоб эти противники были из-за угла, из-за забора, с улицы, в деревне, в своем городе, чтобы они не могли жить в собственном доме, чтобы не могли ни в какой мере внять нашему пожеланию.
. я верю, что в этом вопросе есть смысл. Но, пожалуй, не было бы в этой работе ни одной правильной, но чрезвычайно полезной, работы, которая бы в корне изменила всю жизнь и которую нужно проводить со всем старанием и любовью. «Отчего бы это» — скажете вы, а именно: «отчего бы это»–то, вы совершенно правы! Но мы опять же не можем с этим согласиться, ибо «отчего бы это»–то, мы еще увидим, но совершенно не в этом роде, а в следующем. В этой работе мы видим, что она только на счет «снижения» (отождествления и т. д. – вместе с тем вы увидите, что речь идет о «несуществе», а не об «снижении

In [18]:
print_generations(prefix=". я ненавижу", max_len=200)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}



. я ненавижу либералов-декабристов-монархов и их «патриотические» «указатели» и отдельные слова-лирики. Я не ненавижу либералов-монофизитов, а в том, что они называют либералов «монархами» и т. и м. в слово (отдельно) «монархи», я вижу не в том, что они думают, а в том, что они видят и говорят (вместе с теми либералами, которые говорят: «монофизиками», «пролетариями», «дворцами» и т. д.). Но они не говорят, а говорят, что они говорят и делают (и на самом деле они говорят о том, что и на самом деле говорят о себе), говорят, что они делают, говорят, что они все делают правильно, ибо они могут только только говорить, но не делать не могут ничего другого, кроме
. я ненавижу их, и я презираю их. Не знаю, сколько времени прошло со времени нашей встречи, но я уже знаю, что они не могут оставаться такими, как до сих пор. Я понимаю, что они не могут больше жить без нас. Я говорю им, что они ничего не поняли, что они будут вечно в рабстве, что они будут терпеть, пока им дадут в долг. Я вижу, что